<a href="https://colab.research.google.com/github/victtows/simulador-substitui-o-de-paginas/blob/main/simulador_substitui%C3%A7%C3%A3o_de_paginas_Jos%C3%A9_Victor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from collections import deque, defaultdict

class EstrategiaSubstituicao:
    def __init__(self, quantidade_nichos):
        self.nichos = {'conteudo': [None] * quantidade_nichos}
        self.desvios = 0

    def verificar_presenca(self, pagina):
        return pagina in self.nichos['conteudo']

    def nicho_ocupado(self):
        return None not in self.nichos['conteudo']

    def alocar_pagina(self, pagina):
        for i in range(len(self.nichos['conteudo'])):
            if self.nichos['conteudo'][i] is None:
                self.nichos['conteudo'][i] = pagina
                return i

    def trocar_pagina(self, idx, pagina):
        self.nichos['conteudo'][idx] = pagina

    def escolher_vitima(self, posicao_atual=None, sequencia=None):
        raise NotImplementedError

    def processar_referencia(self, pagina, posicao_atual=None, sequencia=None):
        if not self.verificar_presenca(pagina):
            self.desvios += 1
            if not self.nicho_ocupado():
                self.alocar_pagina(pagina)
            else:
                idx = self.escolher_vitima(posicao_atual, sequencia)
                self.trocar_pagina(idx, pagina)
        self.atualizar_metadados(pagina, posicao_atual, sequencia)

    def atualizar_metadados(self, pagina, posicao_atual=None, sequencia=None):
        pass

    def obter_desvios(self):
        return self.desvios

    def estado_atual(self):
        return list(self.nichos['conteudo'])

class EstrategiaFIFO(EstrategiaSubstituicao):
    def __init__(self, quantidade_nichos):
        super().__init__(quantidade_nichos)
        self.fila = deque()

    def escolher_vitima(self, posicao_atual=None, sequencia=None):
        vitima = self.fila.popleft()
        return self.nichos['conteudo'].index(vitima)

    def atualizar_metadados(self, pagina, posicao_atual=None, sequencia=None):
        if pagina in self.nichos['conteudo'] and pagina not in self.fila:
            self.fila.append(pagina)

class EstrategiaLRU(EstrategiaSubstituicao):
    def __init__(self, quantidade_nichos):
        super().__init__(quantidade_nichos)
        self.ultimo_uso = {}
        self.tempo = 0

    def escolher_vitima(self, posicao_atual=None, sequencia=None):
        presentes = {p: t for p, t in self.ultimo_uso.items() if p in self.nichos['conteudo']}
        if not presentes:
            return 0
        pagina_vitima = min(presentes, key=presentes.get)
        return self.nichos['conteudo'].index(pagina_vitima)

    def atualizar_metadados(self, pagina, posicao_atual=None, sequencia=None):
        self.tempo += 1
        if pagina in self.nichos['conteudo']:
            self.ultimo_uso[pagina] = self.tempo
        else:
            self.ultimo_uso.pop(pagina, None)

class EstrategiaOtima(EstrategiaSubstituicao):
    def __init__(self, quantidade_nichos, sequencia):
        super().__init__(quantidade_nichos)
        self.futuro = defaultdict(deque)
        for i, p in enumerate(sequencia):
            self.futuro[p].append(i)

    def escolher_vitima(self, posicao_atual=None, sequencia=None):
        dist = {}
        for pagina in self.nichos['conteudo']:
            fila = self.futuro.get(pagina, deque())
            while fila and fila[0] <= posicao_atual:
                fila.popleft()
            dist[pagina] = fila[0] if fila else float('inf')
        pagina_pior = max(dist, key=dist.get)
        return self.nichos['conteudo'].index(pagina_pior)

    def atualizar_metadados(self, pagina, posicao_atual=None, sequencia=None):
        if pagina in self.futuro:
            fila = self.futuro[pagina]
            if fila and fila[0] == posicao_atual:
                fila.popleft()

def simular_com_registro(StrategyClass, frames, sequencia):
    if StrategyClass is EstrategiaOtima:
        estrategia = StrategyClass(frames, sequencia)
    else:
        estrategia = StrategyClass(frames)
    estados = []
    for i, pagina in enumerate(sequencia):
        estrategia.processar_referencia(pagina, posicao_atual=i, sequencia=sequencia)
        if isinstance(estrategia, EstrategiaFIFO):
            estrategia.atualizar_metadados(pagina, posicao_atual=i, sequencia=sequencia)
        estados.append(estrategia.estado_atual())
    return estados, estrategia.obter_desvios()


def imprimir_planilha(estados, sequencia, titulo):
    frames = len(estados[0]) if estados else 0
    passos = len(sequencia)
    col_cells = [str(x) for x in sequencia]
    w = max(3, max(len(s) for s in col_cells) + 1)
    print("\n" + titulo)
    cab = "       "
    for s in col_cells:
        cab += s.rjust(w)
    print(cab)
    print("    " + "-" * (w * passos))
    for q in range(frames):
        linha = f"Q{q+1} |"
        for estado in estados:
            v = estado[q]
            linha += (str(v) if v is not None else "-").rjust(w)
        print(linha)

print("simulador substituição de paginas    \nAluno: José Victor Justino Silva4")
print("================================================================= \n")
frames = int(input("Número de quadros: ").strip())
seq = list(map(int, input("Sequência de referências (ex: 1 2 3 4 5 6 7 8 9): ").split()))
mostrar = input("Mostrar estado a cada acesso? (s/n): ").strip().lower() == "s"

est_fifo, faltas_fifo = simular_com_registro(EstrategiaFIFO, frames, seq)
est_lru, faltas_lru = simular_com_registro(EstrategiaLRU, frames, seq)
est_opt, faltas_opt = simular_com_registro(EstrategiaOtima, frames, seq)

if mostrar:
    imprimir_planilha(est_fifo, seq, "=== FIFO ===")
    imprimir_planilha(est_lru, seq, "=== LRU ===")
    imprimir_planilha(est_opt, seq, "=== ÓTIMO (OPT) ===")

print("\nFaltas totais:")
print("FIFO  ->", faltas_fifo)
print("LRU   ->", faltas_lru)
print("ÓTIMO ->", faltas_opt)


simulador substituição de paginas    
Aluno: José Victor Justino Silva4

Número de quadros: 3
Sequência de referências (ex: 1 2 3 4 5 6 7 8 9): 1 2 3 4 5 6 7 8 9
Mostrar estado a cada acesso? (s/n): s

=== FIFO ===
         1  2  3  4  5  6  7  8  9
    ---------------------------
Q1 |  1  1  1  4  4  4  7  7  7
Q2 |  -  2  2  2  5  5  5  8  8
Q3 |  -  -  3  3  3  6  6  6  9

=== LRU ===
         1  2  3  4  5  6  7  8  9
    ---------------------------
Q1 |  1  1  1  4  4  4  7  7  7
Q2 |  -  2  2  2  5  5  5  8  8
Q3 |  -  -  3  3  3  6  6  6  9

=== ÓTIMO (OPT) ===
         1  2  3  4  5  6  7  8  9
    ---------------------------
Q1 |  1  1  1  4  5  6  7  8  9
Q2 |  -  2  2  2  2  2  2  2  2
Q3 |  -  -  3  3  3  3  3  3  3

Faltas totais:
FIFO  -> 9
LRU   -> 9
ÓTIMO -> 9
